In [1]:
!pip install -q transformers huggingface_hub
!pip install -q --upgrade accelerate
!pip install -q -U bitsandbytes
!pip install -q torch

In [2]:
import pandas as pd

# Define the bucket and file names
bucket_name = 'agentsum'  # Replace with your bucket name
soap = f's3://{bucket_name}/sample_summary.csv'

# Load the files
soap = pd.read_csv(soap)

# Display the data
soap.head(99)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:294: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,input,output
0,"Good afternoon, champ, how you holding up? Goo...","Subjective:\n- Symptoms: Lower back pain, radi..."
1,"What brings you in here today? Hi, I'm um, I'm...",Subjective:\n- Presenting with dry cough for 1...
2,Do you have any known allergies to medications...,Subjective:\n- No known allergies to medicatio...
3,"How may I help you today? Yeah I've had, a fev...",Subjective:\n- Fever and dry cough started 4 d...
4,It sounds like that you're experiencing some c...,Subjective:\n- Presenting with chest pain for ...
...,...,...
94,"What brings you in? Hi. Uh, I've just had this...",Subjective:\n- Cough for the past week\n- Pers...
95,"Good morning, ma'am. Oh, good morning, doctor....",Subjective:\n- Symptoms: Difficulty using stai...
96,Are you aware of any medical problems in your ...,Subjective:\n- No medical problems reported in...
97,What's going on with you? What brings you here...,Subjective:\n- Symptoms: Loose watery stools\n...


In [3]:
from huggingface_hub import login
from huggingface_hub import whoami
import getpass

# Prompt the user for the Hugging Face token at runtime
hf_token = getpass.getpass("Enter your Hugging Face token: ")

# Log in using the provided token
login(token=hf_token)


print(whoami(token=hf_token))

Enter your Hugging Face token:  ········


{'type': 'user', 'id': '6644f24fe7ae8316ebf3fee4', 'name': 'LizaPiya', 'fullname': 'Fahmida Liza Piya', 'email': 'lizapiya@udel.edu', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/d43d60b3eba464c3f9b44c34e43b64d6.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'Clinical Note LLama', 'role': 'write', 'createdAt': '2024-06-03T19:29:07.142Z'}}}


In [4]:
!pip install -q sentencepiece

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Load model and tokenizer - CHANGE 1: Model name (kept the same)
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Medical dialogue summarization function - CHANGE 2: Updated for SOAP dialogue
def generate_summary(dialogue):
    # CHANGE 3: Updated prompt for medical dialogue and SOAP format
    prompt = f"""You are a medical expert. Analyze the following patient-doctor dialogue and provide a concise medical summary in SOAP format (Subjective, Objective, Assessment, Plan):\n\n{dialogue}\n\nSummary:"""
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,  # CHANGE 4: Adjusted for dialogue summarization
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "").strip()

# Apply to SOAP dataset - CHANGE 5: Updated function name and structure
def process_soap_data(df):
    results = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating summaries"):
        try:
            summary = generate_summary(row['input'])
            summary_tokens = len(tokenizer.encode(summary))
            input_tokens = len(tokenizer.encode(row['input']))
            target_tokens = len(tokenizer.encode(row['output']))  # CHANGE 6: 'output' instead of 'target'
            
            results.append({
                'sample_id': idx,  # CHANGE 7: sample_id instead of note_id
                'original_input': row['input'],
                'generated_summary': summary,
                'target_summary': row['output'],  # CHANGE 8: 'output' column from SOAP
                'summary_token_count': summary_tokens,
                'input_tokens': input_tokens,
                'target_tokens': target_tokens
            })
        except Exception as e:
            print(f"Error processing sample {idx}: {str(e)}")
            results.append({
                'sample_id': idx,
                'original_input': row['input'],
                'generated_summary': "ERROR: Could not generate summary",
                'target_summary': row['output'],
                'summary_token_count': 0,
                'input_tokens': len(tokenizer.encode(row['input'])) if pd.notna(row['input']) else 0,
                'target_tokens': len(tokenizer.encode(row['output'])) if pd.notna(row['output']) else 0
            })
    
    return pd.DataFrame(results)

# Run the pipeline - CHANGE 9: Updated for SOAP dataset
print("Loading and processing SOAP dataset...")
print(f"Total samples to process: {len(soap)}")  # CHANGE 10: 'soap' instead of 'mimic_iv_bhc_100'

full_results = process_soap_data(soap)  # CHANGE 11: Updated function call

# CHANGE 12: Update output filename for SOAP
full_results.to_csv('mistral_7b_summaries_soap_dataset.csv', index=False)
print(f"\nAll results saved to 'mistral_7b_summaries_soap_dataset.csv'")

# Display some basic statistics
print(f"\nDataset Statistics:")
print(f"Total samples processed: {len(full_results)}")
print(f"Successful generations: {len(full_results[full_results['generated_summary'] != 'ERROR: Could not generate summary'])}")
print(f"Failed generations: {len(full_results[full_results['generated_summary'] == 'ERROR: Could not generate summary'])}")

# Display average token counts
successful_results = full_results[full_results['generated_summary'] != 'ERROR: Could not generate summary']
if len(successful_results) > 0:
    print(f"\nAverage token counts:")
    print(f"Input tokens: {successful_results['input_tokens'].mean():.1f}")
    print(f"Target tokens: {successful_results['target_tokens'].mean():.1f}")
    print(f"Generated summary tokens: {successful_results['summary_token_count'].mean():.1f}")

# Show a sample result
if len(successful_results) > 0:
    print(f"\nSample result:")
    sample = successful_results.iloc[0]
    print(f"Input (first 200 chars): {sample['original_input'][:200]}...")
    print(f"Generated summary: {sample['generated_summary']}")
    print(f"Target summary: {sample['target_summary']}")

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Loading and processing SOAP dataset...
Total samples to process: 100


Generating summaries: 100%|██████████| 100/100 [11:25<00:00,  6.85s/it]


All results saved to 'mistral_7b_summaries_soap_dataset.csv'

Dataset Statistics:
Total samples processed: 100
Successful generations: 100
Failed generations: 0

Average token counts:
Input tokens: 486.7
Target tokens: 191.3
Generated summary tokens: 348.2

Sample result:
Input (first 200 chars): Good afternoon, champ, how you holding up? Good afternoon, Doctor, I have a lot of lower back pain. Oh no, before we begin, how old are you, sir and which hand do you write with? I'm seventy five now....
Generated summary: Subjective:
- Patient is a 75-year-old male who has been experiencing lower back pain for 10 days, with radiating pain down both legs for the past 6 days, causing severe weakness and difficulty walking. The pain started on December 3, 1995. The patient has seen two doctors and received anti-inflammatory medication, but the symptoms have worsened. There is leg numbness since December 11.

Objective:
- No objective data is provided in the dialogue.

Assessment:
- The patient 

### Traditional Metrics¶

In [6]:
!pip install -q nltk bert-score
!pip install -q rouge-metric

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# Option 1: Run directly
%run Soap_Mistral.py

### llm as a judge

In [1]:
%run llm_as_a_judge_mistral.py

Testing single Mistral SOAP sample...
🧪 Testing single Mistral SOAP sample evaluation...
Testing sample 0 from 100 total samples
Sample ID: 0
Summary token count: 200


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



📄 Source length: 1701 characters
📝 Summary length: 771 characters
📄 Source preview: Good afternoon, champ, how you holding up? Good afternoon, Doctor, I have a lot of lower back pain. Oh no, before we begin, how old are you, sir and which hand do you write with? I'm seventy five now....
📝 Summary preview: Subjective:
- Patient is a 75-year-old male who has been experiencing lower back pain for 10 days, with radiating pain down both legs for the past 6 days, causing severe weakness and difficulty walking. The pain started on December 3, 1995. The patient has seen two doctors and received anti-inflammatory medication, but the symptoms have worsened. There is leg numbness since December 11.

Objective:
- No objective data is provided in the dialogue.

Assessment:
- The patient may be experiencing symptoms of a herniated disc or spinal stenosis, given the radiating lower back pain, leg weakness, and numbness. The absence of findings on imaging studies could suggest a misdiagnosis or an in


Does the test look good? Run full evaluation? (y/n):  y



Running full Mistral SOAP evaluation...
🔄 Loading Llama 3 8B model as judge...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

📄 Loaded 100 Mistral-7B generated SOAP summaries
📋 Columns: ['sample_id', 'original_input', 'generated_summary', 'target_summary', 'summary_token_count', 'input_tokens', 'target_tokens']
🔍 Evaluating Mistral SOAP summaries for hallucinations...


Evaluating: 100%|██████████| 100/100 [07:53<00:00,  4.73s/it]


📊 MISTRAL-7B SOAP HALLUCINATION EVALUATION RESULTS:
Metric                    Mean ± Std      Min    Max    Perfect Scores
----------------------------------------------------------------------
Hallucination (1-5)       2.23 ± 0.58   1.0    3.0    8/100
Factual Consistency (1-5) 3.74 ± 0.50   2.0    5.0    2/100
Completeness (1-5)        2.99 ± 0.30   2.0    5.0    1/100
Coherence (1-5)           4.05 ± 0.98   3.0    5.0    50/100

📋 BASELINE QUALITY INSIGHTS:
• High hallucination (≥4): 0/100 (0.0%)
• Low factual consistency (≤2): 1/100 (1.0%)
• Good completeness (≥4): 2/100 (2.0%)
• Good coherence (≥4): 55/100 (55.0%)

📊 FOR BASELINE TABLE:
Hallucination: 2.23 ± 0.58
Factual Consistency: 3.74 ± 0.50
Completeness: 2.99 ± 0.30
Coherence: 4.05 ± 0.98

💾 Results saved to: mistral_soap_judge_results.csv
